# LangChain Simple Chat Model (Azure)

In [ ]:
! pip3 install openai
! pip3 install langchain

In [ ]:
AZURE_OPENAI_API_VERSION = ""
AZURE_OPENAI_API_KEY = ""
AZURE_OPENAI_API_BASE = ""
AZURE_DEPLOYMENT_NAME = ""

In [8]:
from agentquest.drivers.mastermind import MasterMindDriver
from agentquest.utils import Action, load_data

Load a generic 4 digits mastermind game

In [9]:
game = load_data('mastermind', '4 digits')[0]

Initialize the prompt and the LLM

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = '''You are tasked to play the Mastermind game. The host chooses a 
number and gives you the amount of digits. You have to gess the correct number
as fast as you can.
The number can contains repetitions.
The number can contain any possible digit between:
0, 1, 2, 3, 4, 5, 6, 7, 8, 9

At each round you provide a number as guess. The hosts provides you 
1. The number of correct digits in the wrong position
2. The number of correct digits in the correct position

The game ends when the hosts outputs 'You Won!'

Carefully choose your strategy. Avoid bruteforce.

Use the following format:
THOUGHT: think about your guess
ACTION: only your guessed number without additional info.
'''

# Initialize the LLM
llm = AzureChatOpenAI(
    openai_api_type='azure',
    openai_api_version=AZURE_OPENAI_API_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    openai_api_base=AZURE_OPENAI_API_BASE,
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0
)

# Initalize the system prompt
chat_template = ChatPromptTemplate.from_messages([
    ("system", prompt),
    ('ai','OK. I\'ll follow your instructions and try my best to solve the task.'),
])
messages = chat_template.format_messages()

Initialize the mastermind driver and get the first observation

In [11]:
driver = MasterMindDriver(game)
obs = driver.reset() # Get the first observation
print(f'OBSERVATION: {obs.output}')

OBSERVATION: Start guessing the 4 digits number.


Start the LLM loop solving the task

In [12]:
from langchain.schema.messages import HumanMessage, AIMessage

step_cnt = 0
while not obs.done and step_cnt<10:
    # Provide the observation to the LLM
    messages.append(HumanMessage(content=obs.output))
    
    # Get the LLM output
    agent_choice = llm(messages).content
    print(agent_choice)
    
    # Provide the choice to the LLM as history
    messages.append(AIMessage(content=agent_choice))
    
    # Extract the LLM guess
    guess = agent_choice.split('ACTION: ')[-1]
    
    # Build the action
    action = Action(action_value=guess)
    
    # Run a mastermind round and get the observation
    obs = driver.step(action)
    print(f'OBSERVATION: {obs.output}')

    step_cnt+=1

THOUGHT: I'll start by guessing 1234. This will give me information about whether any of these numbers are in the secret number and if any are in the correct position.
ACTION: 1234
OBSERVATION: Your guess has 1 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
THOUGHT: This means that one of the numbers 1, 2, 3, or 4 is in the secret number, but not in the position I guessed. I'll keep the number 1 in my guess but change its position, and replace the other numbers with new ones. I'll guess 1567 next.
ACTION: 1567
OBSERVATION: Your guess has 2 correct numbers in the wrong position and 0 correct numbers in the correct position. Keep guessing.
THOUGHT: This means that two of the numbers 1, 5, 6, or 7 are in the secret number, but not in the positions I guessed. I'll keep the numbers 1 and 5 in my guess but change their positions, and replace the other numbers with new ones. I'll guess 5189 next.
ACTION: 5189
OBSERVATION: Your guess has 3 c